In [31]:
#These are the libraries you can use.  You may add any libraries directy related to threading if this is a direction
#you wish to go (this is not from the course, so it's entirely on you if you wish to use threading).  Any
#further libraries you wish to use you must email me, james@uwaterloo.ca, for permission.

from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
import random
from datetime import datetime
from scipy.optimize import minimize
import time
from math import comb

## Group Assignment
### Team Number: 12
### Team Member Names: Sharuga, Derek, Alex
### Team Strategy Chosen: Market Meet

Disclose any use of AI for this assignment below (detail where and how you used it).  Please see the course outline for acceptable uses of AI.

CHECK DELIST:
- Purpose: checks that there is actually valid market data for this stock
- Code: if 'Close' not in data.columns or data['Close'].isnull().all():

PORTFOLIO GENERATOR
- Purpose: calculate how many shares of each stock to buy, and how much to pay in fees
- Code:

portfolio_value = 1000000 * weights  # Portfolio value based on optimized weights

shares = portfolio_val / np.array([ticker_prices[ticker] for ticker in selected_tickers])  # Number of shares

fee = np.minimum(3.95, 0.001 * shares)

portfolio_val -= fee


In [32]:
#October 01, 2023 to September 30, 2024, 2023

START_DATE = '2023-10-01'
END_DATE = '2024-11-20'

 # date on which we will take stock Close prices
TARGET_DATE = pd.to_datetime('2024-10-22').date()

INVESTMENT = 1000000

CSV_FILE = "Tickers_Example.csv"

In [37]:
# read in CSV
csv = pd.read_csv(CSV_FILE, header=None)

display(csv)

,0
0,AAPL
1,ABBV
2,ABT
3,ACN
4,AGN
5,AIG
6,AMZN
7,AXP
8,BA
9,BAC


In [51]:
# filter out invalid stocks
def validity(tickers):

    # list to keep all of the valid stocks
    final_list = []

    for ticker in tickers:
        #only append the stock to the final stock list if it is isn't delisted, has a US or CAD currency, and has the required monthly share volume
        if (not check_delist(ticker) and
            check_currency(ticker) and
            check_volume(ticker)):
                final_list.append(ticker)
    return final_list

# checks if the consumed Ticker is delisted
def check_delist(ticker):
    stock = yf.Ticker(ticker)
    try:
        data = stock.history(period='1d')
        time.sleep(0.3)
        if data.empty:
            #if we can't find any data on the stock, it's delisted
            return True
        else:
            #check that there is actually valid market data for this stock
            if 'Close' not in data.columns or data['Close'].isnull().all():
                return True
            return False
    except Exception as e:
        #if there is an error in finding the stock's data, we can assume that it's delisted
        return True

# checks if the consumed Ticker meets the requirement of an average monthly share volume of 100,000 shares
def check_volume(ticker):
    volume = yf.Ticker(ticker).history(start=START_DATE, end='2024-09-30')['Volume']
    time.sleep(0.3)
    avg_monthly_volume = volume.resample('ME').mean()
    return avg_monthly_volume.mean() >= 100000

# checks whether or not the consumed Ticker is listed in CAD or in USD
def check_currency(ticker):
    stock = yf.Ticker(ticker)
    try:
        currency = stock.info.get('currency', None)
        return currency in ['USD', 'CAD']
    except Exception as e:
        return False  # if there's an error retrieving the data, we can assume it doesnt satisfy the required monthly volume

# create a new DataFrame only containing the valid Tickers in the consumed CSV
tickers_raw = validity(csv[0])
tickers_raw

$AGN: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
$CELG: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
$MON: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
$RTN: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")


['AAPL',
 'ABBV',
 'ABT',
 'ACN',
 'AIG',
 'AMZN',
 'AXP',
 'BA',
 'BAC',
 'BB.TO',
 'BIIB',
 'BK',
 'BLK',
 'BMY',
 'C',
 'CAT',
 'CL',
 'KO',
 'LLY',
 'LMT',
 'MO',
 'MRK',
 'PEP',
 'PFE',
 'PG',
 'PM',
 'PYPL',
 'QCOM',
 'RY.TO',
 'SHOP.TO',
 'T.TO',
 'TD.TO',
 'TXN',
 'UNH',
 'UNP',
 'UPS',
 'USB']

In [52]:
def days_to_drop(markets):

    markets.dropna(inplace=True)
    months = markets.resample('MS').bfill().index
    
    dates = []

    for month in months:
        temp = markets[(markets.index.month==month.month) & (markets.index.year==month.year)].index
        if len(temp) < 18:
            dates.extend(temp)
    
    return dates

In [69]:
tickers_raw_df = pd.DataFrame()

# calculate and store all returns for all all Tickers
for ticker in tickers_raw:
    data = yf.Ticker(ticker).history(start=START_DATE, end=END_DATE)['Close'].pct_change()
    data.drop(index=data.index[0], inplace=True)
    time.sleep(0.3)
    tickers_raw_df[ticker] = data  # this code calculates daily returns

tickers_raw_df.index = pd.to_datetime(tickers_raw_df.index.strftime("%Y-%m-%d"))

tickers_raw_df

,AAPL,ABBV,ABT,ACN,AIG,AMZN,AXP,BA,BAC,BB.TO,...,QCOM,RY.TO,SHOP.TO,T.TO,TD.TO,TXN,UNH,UNP,UPS,USB
Date,,,,,,,,,,,,,,,,,,,,,
2023-10-03,-0.007770,-0.005531,0.000627,-0.006585,-0.023480,-0.036614,-0.029636,0.005803,-0.029588,-0.006645,...,-0.008191,-0.024235,-0.046045,0.006083,-0.009452,-0.016880,-0.009911,0.005626,-0.008576,-0.024891
2023-10-04,0.007309,0.001764,-0.001462,0.010319,0.012278,0.018281,0.010893,-0.011592,0.001158,-0.020067,...,-0.003086,0.009281,0.025412,0.032093,-0.002636,0.004579,0.002257,-0.007410,-0.000195,-0.003191
2023-10-05,0.007198,-0.001625,0.005750,0.000873,0.013309,-0.008189,0.007259,-0.002356,0.004626,-0.112628,...,-0.000455,0.007706,-0.009968,0.019378,0.011595,-0.018105,0.010987,-0.008058,-0.002212,0.020167
2023-10-06,0.014751,0.005358,0.007069,0.008170,0.011970,0.015878,0.015161,0.005851,0.000384,-0.009615,...,0.009836,-0.001477,0.024612,-0.003095,0.005164,0.012185,0.016621,0.009071,0.005803,-0.000314
2023-10-09,0.008451,0.005869,-0.001239,-0.000576,0.003614,0.002344,-0.003348,0.005924,0.009206,NaN,...,0.008297,NaN,NaN,NaN,NaN,-0.001784,0.003239,0.009088,0.000454,0.004708
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-13,0.003969,-0.004325,-0.006012,0.026649,0.016431,0.024843,-0.003015,-0.035820,0.000218,0.014793,...,-0.018298,-0.001852,0.058715,-0.003199,0.002936,-0.033180,-0.014317,0.006279,0.027201,0.008077
2024-11-14,0.013770,-0.004227,-0.006134,-0.022014,-0.004468,-0.012237,0.002294,-0.013074,0.000654,-0.011662,...,0.022129,-0.002204,-0.048201,-0.021550,0.013747,0.001361,-0.020995,-0.020091,-0.007780,-0.016807
2024-11-15,-0.014109,-0.027354,0.007563,-0.023476,0.000264,-0.041942,-0.004960,0.014840,0.018518,-0.017699,...,-0.021162,-0.005347,-0.003650,0.002343,-0.010547,-0.023689,-0.001551,-0.000042,0.010781,-0.008149


In [71]:
def std_rank(returns):
    std_devs = returns.std()
    
    # Create a DataFrame with tickers and their respective standard deviations
    ranked_stocks = pd.DataFrame({
        "ticker": std_devs.index,
        "STD": std_devs.values
    })
    
    # Sort by standard deviation in ascending order (lowest STD at the top)
    ranked_stocks = ranked_stocks.sort_values(by="STD", ascending=True)
    
    return ranked_stocks['ticker'].values[:24].tolist()

ticker_lst = std_rank(tickers_raw_df)
print(ticker_lst)

['KO', 'RY.TO', 'PG', 'CL', 'T.TO', 'PEP', 'TD.TO', 'BK', 'LMT', 'ABT', 'UNP', 'MO', 'BLK', 'PM', 'MRK', 'AIG', 'AAPL', 'ABBV', 'ACN', 'UNH', 'AXP', 'BAC', 'PFE', 'BIIB']


In [73]:

index_lst = ["XIU.TO","^GSPC"]

exch_rate = yf.download("CADUSD=x",start="2024-11-20", end="2024-11-21")["Close"].iloc[0] #MUST BE CHANGED TO PULL NOVEMBER 22ND

#Takes a yf.Ticker "ticker", checks if the ticker is in canadian dollars. If so, it will just do a simple API pull for the price history.
#If the price is listed in USD, a conversion operation will simply be applied to each price.
def yfin_pull_convert_USD(ticker):
    listed_currency = ticker.fast_info["currency"]
    if listed_currency == "CAD":
        return ticker.history(start=START_DATE, end=END_DATE, interval="1d")["Close"]
    elif listed_currency == "USD":
        return ticker.history(start=START_DATE, end=END_DATE, interval="1d")["Close"] * (1/exch_rate)
    raise ValueError("Currency not in CAD or USD")

#Test cases
#display(yf.Ticker("NVDA").history(start=START_DATE, end=END_DATE, interval="1d")["Close"]
#yfin_pull_convert_USD(yf.Ticker("NVDA"))

#Main function to store the stock prices of the ticker
def gen_tickers(tlist, ilist):
    ret_dataframe = pd.DataFrame()
    check_month_dataframe = pd.DataFrame()
    for i_str in ilist:
        index_ticker = yf.Ticker(i_str)
        check_month_dataframe[i_str] = yfin_pull_convert_USD(index_ticker)
        ret_dataframe[i_str] = yfin_pull_convert_USD(index_ticker)
    
    for t_str in tlist:
        ticker = yf.Ticker(t_str)
        ret_dataframe[t_str] = yfin_pull_convert_USD(ticker)
    ret_dataframe.drop(days_to_drop(check_month_dataframe), inplace=True)
    return ret_dataframe

#Function call: stores a dataframe of index values and stock values
stock_values = gen_tickers(ticker_lst,index_lst)
# display(stock_values)

[*********************100%***********************]  1 of 1 completed


In [21]:
market_label = "Market Returns"
#Function that transforms a list of values into a list of percent returns.
#Also adds a column of the simple average returns of the index tickers.
def convert_pct_returns(stock_value_df):
    ret_dataframe = stock_value_df.interpolate().pct_change()
    ret_dataframe.dropna(inplace=True)
    ret_dataframe[market_label] = ret_dataframe[index_lst].mean(axis=1)
    df_col_order = index_lst + ["Market Returns"] + ticker_lst
    ret_dataframe = ret_dataframe[df_col_order]
    return ret_dataframe

stock_returns = convert_pct_returns(stock_values)
display(stock_returns)



,XIU.TO,^GSPC,Market Returns,AAPL
Date,,,,
2023-10-03 00:00:00-04:00,-0.008913,-0.013744,-0.011329,-0.007770
2023-10-04 00:00:00-04:00,0.001384,0.008110,0.004747,0.007309
2023-10-05 00:00:00-04:00,0.005872,-0.001304,0.002284,0.007198
2023-10-06 00:00:00-04:00,0.004464,0.011815,0.008140,0.014751
2023-10-10 00:00:00-04:00,0.014359,0.011545,0.012952,0.005071
...,...,...,...,...
2024-10-25 00:00:00-04:00,-0.003760,-0.000299,-0.002030,0.003643
2024-10-28 00:00:00-04:00,0.005392,0.002651,0.004022,0.008599
2024-10-29 00:00:00-04:00,-0.001341,0.001614,0.000137,0.001157


In [22]:
#Creating the correlation matrix: takes a dataframe of values and a string of columns to drop to make the correlation matrix
def correl(data, dropvalue):
    data_marketdrop = data.drop(labels=dropvalue, axis=1)
    ret_corr = data_marketdrop.corr()
    return ret_corr

stock_correlations = correl(stock_returns, [market_label] + index_lst)
#Apply a format to the DataFrame
display(stock_correlations.style.background_gradient(cmap='RdYlGn_r'))


,AAPL
AAPL,1.000000


In [23]:
#finding the ticker coordinates of the lowest correlation value (AI)
def low_correl_ticker_pairs(correlation_matrix):
    if correlation_matrix.shape[0] > 1:
        correl_pairs = correlation_matrix.unstack()
        correl_pairs = correl_pairs[correl_pairs.index.get_level_values(0) != correl_pairs.index.get_level_values(1)]
        lowest_corr_pair = correl_pairs.idxmin()
        lowest_corr_value = correl_pairs.min()
        return list(lowest_corr_pair)
    else:
        remaining_column_ticker = correlation_matrix.columns[0]
        return [remaining_column_ticker]

#recursing through the entire correlation matrix to extract every correlation pair
def corr_pair_extract(correl_matrix):
    #creating a list to store low-correlation pairs into
    low_corr_pairs = []
    while correl_matrix.shape[0] > 0:
        pair = low_correl_ticker_pairs(correl_matrix)
        #removing the tickers after they are packaged into a correlation pair
        if isinstance(pair, str):
            ticker1 = pair
            correl_matrix = correl_matrix.drop(ticker1, axis=0)
            correl_matrix = correl_matrix.drop(ticker1, axis=1)
        else: 
            low_corr_pairs.append(pair)
            correl_matrix = correl_matrix.drop(pair, axis=0)
            correl_matrix = correl_matrix.drop(pair, axis=1)
    return low_corr_pairs

#running the correlation pairing function and storing it in variable "stock pairs"
stock_pairs = corr_pair_extract(stock_correlations)
print(stock_pairs)

[['AAPL']]


In [24]:
#Arbitrarily assign a seed to keep results consistent
random.seed(21152764)

#This part creates lists of stocks, ranging from sizes 12 to 24, 5 of each size, which randomly selects stocks.
def make_port_list(stock_pairs):
    ret_list = []
    #iterates through the minimum selectable stock pairs (6) and the maximum (length of the stock pairs)
    print(len(stock_pairs))
    for i in range(6, len(stock_pairs)+1):
        print(i)
        #simple iteration, of 5 times
        for j in range(min(comb(len(stock_pairs), i), 5)):
            #generates a random list of indexes to pick from the stock pairs
            rand_list = random.sample(range(0,len(stock_pairs)), i)
            portfolio = []
            for k in rand_list:
                portfolio += stock_pairs[k]
            ret_list.append(portfolio)
    return ret_list

port_list = make_port_list(stock_pairs)

1


In [25]:
stock_values_just_date = stock_values
stock_values_just_date.index = pd.to_datetime(stock_values_just_date.index)
stock_values_just_date.index = stock_values_just_date.index.date

# Display the updated DataFrame
display(stock_values_just_date)


,XIU.TO,^GSPC,AAPL
2023-10-02,28.043184,5984.447960,240.968586
2023-10-03,27.793228,5902.197278,239.096301
2023-10-04,27.831682,5950.062652,240.843763
2023-10-05,27.995119,5942.303590,242.577342
2023-10-06,28.120094,6012.511272,246.155482
...,...,...,...
2024-10-25,36.783108,8105.230979,322.577733
2024-10-28,36.981453,8126.721541,325.351720
2024-10-29,36.931870,8139.839103,325.728085
2024-10-30,36.872364,8112.975730,320.751633


In [40]:
portfolio_data = []

def portfolio_generator():
    global portfolio_data

    # creating portfolios -- starting wtih 12 stocks, going upto 24 stocks
    for i in range(len(port_list)):

        # for the ith portfolio, select the first i tickers
        selected_tickers = port_list[i]
        
        # objective function -- will be used in optimization steps
        # returns the tracking error between the portfolio's returns and the market's returns
        def objective(weights):
            return np.std(np.dot(stock_returns[selected_tickers].values, weights) - stock_returns['Market Returns'].values)
            # return np.std(np.dot(stock_returns[selected_tickers].mean(), weights) - stock_returns['Market Returns'].dropna().mean())
        
        # initial weights are calcualted so that, to start, every stock has equal weight
        initial_weights = [1 / len(selected_tickers)] * len(selected_tickers)
        
        # this constraint ensures that all of the stock weights sum to 1 (100%)
        def sum_weights(weights):
            return sum(weights) - 1
        
        constraints = [{'type': 'eq', 'fun': sum_weights}]
        
        # the bounds for each stock are that each stock must have a minimum weight of 1/2n% and maximum weight of 15%
        bounds = [(1 / (2 * len(selected_tickers)), 0.15)] * len(selected_tickers)
        
        # optimize the portfolio weights
        result = minimize(objective, initial_weights, constraints=constraints, bounds=bounds)
        
        if result.success:

            weights = result.x
                        
            # calculate portfolio value and the number of shares to purchase
            portfolio_value = 1000000 * weights
            shares = portfolio_value / np.array([stock_values_just_date.at[TARGET_DATE, ticker] for ticker in selected_tickers]) 

            # calculate fees
            fee = np.minimum(3.95, 0.001 * shares)
            
            # subtract fees from portfolio value
            portfolio_value -= fee
            
            # Add to the results
            portfolio_data.append({
                'tickers': (selected_tickers),  # Tickers in the portfolio
                'weights': weights,  # Optimized weights
                'Tracking Error': result.fun,  # Standard deviation (risk)
                'Difference of Mean Returns': abs(np.dot(stock_returns[selected_tickers].values, weights).mean() - stock_returns['Market Returns'].values.mean()),
                'num_stocks': i,  # Number of stocks in the portfolio
                'total_cost': portfolio_value.sum() + sum(fee),  # Total cost after fees
                'shares_purchased': shares,  # Shares purchased
                'portfolio_value': portfolio_value.sum(),  # Total portfolio value after fees
                'final_fees': sum(fee)
            })

    return pd.DataFrame(portfolio_data)

portfolio_df = portfolio_generator()
# portfolio_df['weights'].iloc[0]

display(portfolio_df)

[]

""


When your code is run with our .csv file, it must create a DataFrame called “Portfolio_Final” where the index starts at 1 and ends at the number of stocks that your code chooses.  The headings must be as follows: Ticker, Price, Currency, Shares, Value, Weight. Ticker will be the ticker your code selected, Price is the price on November 25, 2023, Currency is either USD or CAD, Shares is the number of shares you purchased of that stock, Value is the total value of those shares, and Weight is the weight that the value of shares represents relative to the value of your portfolio (which again, must be $1,000,000 CAD).  You should show that your total adds to $1,000,000 CAD and that the weights add to 100%.  This DataFrame must be printed to the screen as the second to last output to the screen.  The final output to the screen will be a declaration to be detailed below.


In [27]:
temp = portfolio_df[portfolio_df['Difference of Mean Returns'] == portfolio_df['Difference of Mean Returns'].min()]


# Define columns for the final portfolio DataFrame
port = []

for i in range(len(temp['tickers'].iloc[0])):
    port.append({
        "Ticker": temp['tickers'].iloc[0][i],
        "Price": stock_values.loc[pd.to_datetime(TARGET_DATE).date(), temp['tickers'].iloc[0][i]],
        "Currency": yf.Ticker(temp['tickers'].iloc[0][i]).info['currency'],
        "Shares": temp['shares_purchased'].iloc[0][i],
        "Value": stock_values.loc[pd.to_datetime(TARGET_DATE).date(), temp['tickers'].iloc[0][i]] * temp['shares_purchased'].iloc[0][i],
        "Weight": temp['weights'].iloc[0][i]})

Portfolio_Final = pd.DataFrame(port)

# Reset index to start at 1
Portfolio_Final.index = Portfolio_Final.index + 1

Portfolio_Final

KeyError: 'Difference of Mean Returns'

In [ ]:
Portfolio_Final.to_csv("Stocks_Group_12.csv", sep='\t', encoding='UTF-8')

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Alex, Derek & Sharuga